<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Herrera Suárez
- Nombre de alumno 2: Hecmar Taucare Reyes


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/Joaquin-HS/MDS7202)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Se carga el dataset con el archivo "online_retail_data.pickle"
df_retail = pd.read_pickle("online_retail_data.pickle")
df_retail

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [4]:
def explore_data(dataframe_in):
    # Se plotea un histograma para las variables numéricas
    for col in dataframe_in.select_dtypes(include=np.number).columns:
        if col == "Customer ID":
            continue
        fig = px.histogram(dataframe_in, x=col, title=f"Histograma de {col}")
        fig.show()

    # Se podría aplicar logaritmo a los valores para visualizar mejor la distribución,
    # pero esto se abordará mediante rango intercuartilico en la siguiente sección.

    # Se imprime un conteo de datos nulos por variable
    print("Conteo de datos nulos por variable")
    print(dataframe_in.isnull().sum())

In [ ]:
explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [6]:
class IQR(BaseEstimator, TransformerMixin):
  def __init__(self, lambd=1.5):
    self.lambd = lambd  # Factor de multiplicación para el rango intercuartil (umbral de outliers)
    self.q1 = None  # Primer cuartil
    self.q3 = None  # Tercer cuartil
    self.iqr = None  # Rango intercuartil
    self.lower_bound = None  # Límite inferior
    self.upper_bound = None  # Límite superior

  def fit(self, X):
    # Se calcula el primer y tercer cuartil
    self.q1 = X.quantile(0.25)
    self.q3 = X.quantile(0.75)

    # Se calcula el rango intercuartil
    self.iqr = self.q3 - self.q1

    # Se calculan los límites inferior y superior
    self.lower_bound = self.q1 - self.iqr * self.lambd
    self.upper_bound = self.q3 + self.iqr * self.lambd

    return self

  def transform(self, X):
    # Se eliminan los valores atípicos del dataframe para cualquier columna
    X = X[(X >= self.lower_bound) & (X <= self.upper_bound)]
    
    return X

  def set_output(self,transform='default'):
    # No se modifica esta función
    return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicando el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definición las variables que pasarán por cada pipeline
numerical_columns = ['Quantity', 'Price']
categorical_columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate', 'Customer ID', 'Country']

# Definición del pipeline
numeric_transformations = Pipeline(steps=[('iqr', IQR(lambd=1.5))])

# ColumnTransformer
column_transformer = ColumnTransformer(transformers=[
                                        ('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', 'passthrough', categorical_columns)
                                        ],
                                        verbose_feature_names_out=False)
column_transformer.set_output(transform='pandas')

# Se aplica ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)

# Gráficos
explore_data(df_retail)
explore_data(df_iqr)

Como se puede observar en los gráficos, aplicar IQR permite eliminar datos anormales, generando así una mejor visualización de la distribución de cierta variable, que es lo que se busca al realizar un estudio inicial, dejando los outliers para ser analizados aparte. Nótese que estos outliers pasan a ser valores nulos según la implementación utilizada, por lo que habría que abordar dicho aspecto según lo que se quiera.

Las distribuciones de las variables numéricas analizadas exhiben que, en el caso de los precios, se tienen distintos valores marcados, con una tendencia a estar más centrados entorno a 1 o 2 unidades. Por otro lado, las cantidades están mucho más agrupadas, acercándose a valores pequeños como 5 o 10 unidades.

Por último, se destaca que un aumento en el valor de lambda genera un mayor recorte de datos, haciendo más estrictos los límites superior e inferior, los cuales indican los registros permitidos. Esto puede generar que se eliminen datos que son parte de la distribución normal de la variable, perdiendo más información de la debida. Se estableció un valor de 1.5 por defecto al ser un valor intermedio. 

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
# 1
categoric_transformations = Pipeline(steps=[
    ('mode_imputer', SimpleImputer(strategy='most_frequent')),
])

# 2
numeric_transformations = Pipeline(steps=[
    ('iqr', IQR(lambd=1.5)),
    ('mean_imputer', SimpleImputer(strategy='mean'))])

# 3
column_transformer = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)
column_transformer.set_output(transform='pandas')

df_mean_imputer = column_transformer.fit_transform(df_retail)

# 4
explore_data(df_mean_imputer)
explore_data(df_iqr)

De los gráficos se observa que la distribución no varía de manera destacable, salvo por la agregación de un punto nuevo en cada histograma, correspondiente al valor promedio de cada variable con una frecuencia igual al número de nulos que se tiene para esa variable (2.28 - 2.29 para precio, y 6.8188 para cantidad). Se menciona, por un lado, lo inapropiado que puede ser realizar este tipo de imputación para variables que solo tienen valores posibles enteros (como la variable cantidad); por otro, lo poco representativo que es simplemente añadir un peak con gran frecuencia si se están analizando distribuciones; y por último, el notorio efecto que tiene el aumento de nulos debido a la aplicación de IQR sobre procedimientos de imputación, ya que generó que los peaks respectivos sean aún mayores que los que se hubieran generado si se eliminaban directamente los outliers (esto no necesariamente es algo inapropiado, ya que depende de lo que desee el analista).

In [ ]:
# 5
numeric_transformations = Pipeline(steps=[
    ('iqr', IQR(lambd=1.5)),
    ('knn_imputer', KNNImputer(n_neighbors=3))])  # Se establece el número de vecinos más cercanos en 3 para evitar tiempos de ejecución demasiado largos

column_transformer = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)
column_transformer.set_output(transform='pandas')

df_knn_imputer = column_transformer.fit_transform(df_retail)

# 6
explore_data(df_knn_imputer)
explore_data(df_iqr)

Las distribuciones de los datos al imputar con KNN tienden a ser más suaves que las distribuciones sin imputar, rellenando espacios intermedios sin perturbar demasiado la distribución. Sin embargo, estos valores imputados no dejan de ser inventados, generando por ejemplo cantidades con valores decimales.

In [ ]:
# 7
explore_data(df_knn_imputer)
explore_data(df_mean_imputer)

El método escogido es KNN, principalmente por su mejor capacidad de mantener la forma de la distribución de las variables, con respecto a cómo lo hace la imputación por media

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [11]:
def custom_features(dataframe_in):
    # Se trabaja con la columna InvoiceDate en formato datetime
    dataframe_in['InvoiceDate'] = pd.to_datetime(dataframe_in['InvoiceDate'])
    
    # Gasto total por registro
    dataframe_in['TotalSpent'] = dataframe_in['Quantity'] * dataframe_in['Price']
    
    # Antes de agrupar se define la fecha actual
    today = dataframe_in['InvoiceDate'].max()

    # Se agrupa por Customer ID
    df_grouped = dataframe_in.groupby('Customer ID')

    # Length (L): Intervalo de tiempo, en días, entre la primera y la última visita del cliente
    L = (df_grouped['InvoiceDate'].max() - df_grouped['InvoiceDate'].min()).dt.days

    # Recency (R): Indica hace cuanto tiempo el cliente realizó su última compra
    R = (today - df_grouped['InvoiceDate'].max()).dt.days

    # Monetary (M): Cantidad media de dinero gastada por cada visita del cliente
    M = df_grouped['TotalSpent'].mean()

    # Frequency (F): Número total de visitas del cliente
    F = df_grouped['InvoiceDate'].nunique()

    # Periodicity (P): Desviación estándar del tiempo entre visitas
    def calc_periodicity(df_grouped):
        # Se ordenan las fechas y se calcula la diferencia en días entre cada visita
        sorted_dates = df_grouped['InvoiceDate'].sort_values()
        visit_intervals = sorted_dates.diff().dt.days.dropna()  # Se eliminan los valores nulos, de existir
        
        # Desviación estándar del tiempo entre visitas
        return visit_intervals.std() if len(visit_intervals) > 1 else np.nan
    
    P = df_grouped.apply(calc_periodicity)

    # Se crea un nuevo dataframe con las variables calculadas
    lrmfp_df = pd.DataFrame({'Length': L, 'Recency': R, 'Monetary': M, 'Frequency': F, 'Periodicity': P}).reset_index()

    return lrmfp_df

In [ ]:
lrmfp_df = custom_features(df_knn_imputer)
lrmfp_df

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:
# 1
retail_pipeline = Pipeline(steps=[
    ('col_transformer', column_transformer),
    ('custom_features', FunctionTransformer(custom_features, validate=False))
])

# 2
df_custom = retail_pipeline.fit_transform(df_retail)

# 3
explore_data(df_custom)

- **Length**: Se destaca que para la mayoría de personas ha pasado entre 0 y 9 días entre la primera y la última visita, lo que indica que la mayoría son clientes nuevos. Por otro lado, existe un peak de frecuencia entre 200 y 250 días, y entre 300 y 350 días, lo que podría indicar un sector fiel de clientes (esto no es necesariamente así, ya que podría darse el caso de que un cliente compra entre periodos largos de tiempo, pero no entre medio. Para un análisis en detalle habría que disolver la agrupación y revisar esos clientes específicos).

- **Recency**: Que el gráfico este enfocado principalmente en un número de días bajo implica que la mayoría de clientes está activo recientemente. Nótese que esto se condice con el hecho de que la mayoría de los clientes son nuevos. Por otro lado, aún que en menor cantidad, existen clientes que hace mucho tiempo hicieron su última compra. Esto quiere decir que, o bien existen clientes que decidieron dejar de comprar en alguna de las tiendas, o bien realizan compras distanciadas en el tiempo.

- **Monetary**: La distribución tiende a ser una mezcla de normales, exhibiendo que existe un grupo de compras de menor valor (entre 5 y 6 unidades), y otro de compras de mayor valor (entre 14 y 15 unidades). Se destaca que el grupo de compras de mayor valor es más frecuente que el grupo de compras de menor valor. Por último, se aprecia que existen clientes outliers que en promedio realizan compras muy superiores en gasto con respecto a las normales mencionadas.

- **Frequency**: Este gráfico señala que la mayoría de clientes no ha visitado más de 5 veces, es decir, no visita mucho las tiendas. Además, esto se condice con lo que se ha mencionado previamente con respecto a que la mayoría son clientes nuevos. Por otro lado, se destaca la existencia, aunque baja, de clientes con un gran número de visitas, yendo desde 50 a incluso más de 200 apariciones.

- **Periodicity**: El gráfico muestra que, en general, la mayoría de los clientes realiza visitas de manera regular (baja variabilidad en los intervalos entre visitas). Por otro lado, existe un grupo importante de clientes que tiene una variabilidad mucho mayor (valor de periodicity entorno a 9 y 30). Esto implica la existencia de dos grupos marcados de clientes: los que compran a menudo, y los que compran entre periodos largos.

**Insight**:  Se puede señalar que existen 4 tipos de clientes: aquellos que compran con cierta regularidad, ya sea gastando más o menos, y los que compran en intervalos más largos de tiempo, de nuevo ya sea gastando más o menos. De esta forma, llevar a cabo políticas comerciales que toman en cuenta distintas propuestas de valor permitiría maximizar beneficios para la empresa, ya sea mediante promociones u ofertas que hagan que los clientes tiendan a gastar más, o que aumenten su regularidad, entre muchas otras.

### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [14]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self, X):
        # Valores mínimos y máximos por columna
        self.min_values = X.min(axis=0)
        self.max_values = X.max(axis=0)

        return self

    def transform(self, X):
        # Transformación Min-Max        
        return (X - self.min_values) / (self.max_values - self.min_values)

    def set_output(self,transform='default'):
        # No se modifica este método
        return self

#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [ ]:
numeric_transformations = Pipeline(steps=[
    ('iqr', IQR(lambd=1.5)),
    ('knn_imputer', KNNImputer(n_neighbors=3)),  # Se establece el número de vecinos más cercanos en 3 para evitar tiempos de ejecución demasiado largos
    ('minmax', MinMax())
])  

column_transformer = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)
column_transformer.set_output(transform='pandas')

df_minmax = column_transformer.fit_transform(df_retail)

explore_data(df_minmax)
explore_data(df_retail)

Los cambios que se exhiben en la distribución corresponden a los mismos vistos en los puntos anteriores con respecto a la eliminación de outliers y la imputación de nulos mediante KNN, pero ahora agregando el hecho de que los valores de las variables se mueven entre 0 y 1 (escalamiento no altera proporciones). Esto es de gran utilidad cuando se quieren realizar comparaciones entre variables que se mueven en distintas escalas, para no darle un mayor peso a alguna sobre la otra, por ejemplo en algoritmos que requieren características normalizadas.

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Los pipelines ayudan en un gran número de aspectos al hacer Feature Engineering. Por un lado, el simplificar y combinar los distintos procesos que se deben aplicar en machine learning en una única entidad, facilitando el flujo de trabajo, reduciendo fuentes de errores y asegurando consistencia entre los datos. Por otro, permite que ajustar hiperparámetros sea una tarea sencilla cuando se quieran realizar pruebas con distintas combinaciones de hiperparámetros para mejorar el rendimiento de cierto modelo. Por último, la simplificación y compactación en la manipulación de datos permite no solo que un código sea mucho más legible y reutilizable con respecto a un proyecto que no utilice pipelines, sino que también asegura una mejora de rendimiento para modelos complejos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>